In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

# Asignar la ruta donde deseas guardar las imágenes
PROJECT_ROOT_DIR = r"C:\Users\ezequ\OneDrive\Documentos\Facultad\Optimizacion\tp1"
CHAPTER_ID = "imagenes"  # El nombre de la capeta donde voy a guardar las imagenes

# Crear la ruta completa donde se guardarán las imágenes
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)



In [3]:
#path = r'C:\Users\ezequ\OneDrive\Documentos\Facultad\Aprendizaje Automatico\material-main\tp\01_aprendizaje_supervisado\datos\data.csv'
#df = pd.read_csv(path)
df = pd.read_csv('data.csv')

In [4]:
df['target'].value_counts()

target
0    343
1    157
Name: count, dtype: int64

La relacion entre clases es 157/343 = 0.46 siendo la mayoritaria la clase representada por el 0 por lo que estamos ante un dataset desbalanceado

A priori hacer un split 400 100 para desarrollo validacion suena razonable , estamos dejando 20% de nuestro dataset para validar.

Capaz es un poco mucho ese porcentaje pero creo que eso aseguraria que los resultados obtenidos tendran un poco mas de robustez.

Dentro de desarrollo capaz podemos hacer algo como k folds con k = 4 para sacarle un poco mas el jugo a los datos.

train = 80%
validacion = 20%

In [5]:
def train_test_split(df,train_size=0.8):
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    n = df.shape[0]
    cantNeg, cantPos = df['target'].value_counts()
    proporcion = cantPos / n

    cant_en_train = int(n*train_size)
    cant_en_validacion = n - cant_en_train

    res = []
    cant_positivas = int(proporcion * cant_en_validacion)
    cant_negativas = cant_en_validacion - cant_positivas

    contador_positivos, contador_negativos = 0, 0
    for index, value in enumerate(df['target']):
        if value == 0:
            contador_negativos += 1
            res.append(index)

        if contador_negativos == cant_negativas:
            break
    for index, value in enumerate(df['target']):

        if value == 1:
            contador_positivos += 1
            res.append(index)

        if contador_positivos == cant_positivas:
            break

    df_dev = df.loc[~df.index.isin(res)].reset_index(drop=True)
    df_val = df.loc[res].reset_index(drop=True)
    return df_dev,df_val

In [6]:
df_dev , df_val =  train_test_split(df)
df_train, df_test = train_test_split(df_dev)

In [7]:
pd.value_counts(df_test['target'])

target
0    55
1    25
Name: count, dtype: int64

# Ejercicio 2

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, roc_auc_score , average_precision_score

In [9]:
X_train = df_train.drop(columns=['target']).values

y_train = df_train['target'].values

X_test  = df_test.drop(columns=['target']).values

y_test  = df_test['target'].values

X_val   = df_val.drop(columns=['target']).values

y_val   =  df_val['target'].values

In [10]:
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=51)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=51)

In [11]:
scoring = ['accuracy', 'roc_auc', 'average_precision']
skf = StratifiedKFold(n_splits=5)

cv_results_train = cross_validate(tree_clf, X_train, y_train, cv = skf, scoring = scoring)

print("Cross-validation results in Train:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_train[f'test_{metric}'].round(3)}")
    print(f"Promedio de {metric.capitalize()}:  {cv_results_train[f'test_{metric}'].mean().round(3)}")
    print('\n')

print("--------------------------------------------------------")

cv_results_test = cross_validate(tree_clf, X_test, y_test, cv = skf, scoring = scoring)

print("Cross-validation results in Test:")
for metric in scoring:
    print(f"{metric.capitalize()}: {cv_results_test[f'test_{metric}'].round(3)}")
    print(f"Promedio de {metric.capitalize()}:  {cv_results_test[f'test_{metric}'].mean().round(3)}")
    print('\n')


Cross-validation results in Train:
Accuracy: [0.609 0.609 0.625 0.734 0.656]
Promedio de Accuracy:  0.647


Roc_auc: [0.637 0.632 0.526 0.741 0.505]
Promedio de Roc_auc:  0.608


Average_precision: [0.416 0.471 0.327 0.553 0.331]
Promedio de Average_precision:  0.42


--------------------------------------------------------
Cross-validation results in Test:
Accuracy: [0.625 0.812 0.5   0.938 0.812]
Promedio de Accuracy:  0.738


Roc_auc: [0.527 0.745 0.527 0.955 0.727]
Promedio de Roc_auc:  0.696


Average_precision: [0.348 0.542 0.325 0.833 0.575]
Promedio de Average_precision:  0.524




Basado en pagina 288 de Introduction to Machine Learning with Python

In [12]:
alturas = [3, 5, None]
criterios = ["gini", "entropy"]

resultados = {}

for altura in alturas:
    for criterio in criterios:

        tree = DecisionTreeClassifier(max_depth = altura, criterion = criterio, random_state = 51)

        tree.fit(X_train, y_train)

        cv_results_train = cross_validate(tree, X_train, y_train, cv = skf, scoring = 'accuracy')

        y_pred_test = tree.predict(X_test)

        accuracy_test = accuracy_score(y_test, y_pred_test)

        global_score_test= cv_results_train['test_score'].mean()

        resultados[(altura, criterio)] = (global_score_test,accuracy_test)

results_df = pd.DataFrame(resultados.values(), index=resultados.keys(), columns=['Train Score', 'Test Score'])



In [13]:
results_df

Train Score  Test Score
3   gini        0.646875      0.6625
    entropy     0.640625      0.7250
5   gini        0.646875      0.6750
    entropy     0.621875      0.7125
NaN gini        0.631250      0.5625
    entropy     0.606250      0.6625

# Ejercicio 3

In [159]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
import scipy


# Hay que ver en la documentarcion del modelo cada parametro y tirar una distribucion con los valores mas probables
# Si pones una lista la toma como uniforme discreta

# Aca algunos me tiraban error asi que los comente
param_dist = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': scipy.stats.randint(1,40), #puede ser float
    'min_samples_split': scipy.stats.randint(2,20), #puede ser float
    'min_samples_leaf': scipy.stats.randint(1,20),
    #'min_weight_fraction_leaf':scipy.stats.uniform(0,1) ,
    'max_features': scipy.stats.randint(1,20),
    'random_state': [42],
    'max_leaf_nodes': scipy.stats.randint(2,len(X_train)),
    #'min_impurity_decrease': scipy.stats.uniform(0,0.1),
    #'class_weight': {i: scipy.stats.uniform(0,1) for i in range(len(X_train[0]))},
    #'ccp_alpha': scipy.stats.uniform(0,0.1),
}


tree = DecisionTreeClassifier()
n_iter_search = 30    # Esto es la cantidad de configuraciones que pruebo
random_search = RandomizedSearchCV(estimator=tree, param_distributions=param_dist, scoring='roc_auc', cv=5, n_iter=n_iter_search)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: DecisionTreeClassifier(max_depth=11, max_features=10, max_leaf_nodes=212,
                       min_samples_leaf=5, min_samples_split=10,
                       random_state=42)
Best AUCROC Score: 0.64186033927313


In [160]:
from sklearn.neighbors import KNeighborsClassifier

param_dist = {
    'n_neighbors': scipy.stats.randint(1, 30),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': scipy.stats.randint(1, 50),
    'p': scipy.stats.randint(1, 5),
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
    #'metric_params':,
    #'n_jobs':,
}

knn = KNeighborsClassifier()
n_iter_search = 30
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, scoring='roc_auc', cv=5, n_iter=n_iter_search)


# random_search.fit(X_train_normalized, y_train)
# Normalizo los datos para knn
from sklearn import preprocessing
X_train_normalized = preprocessing.normalize(X_train.T).T
random_search.fit(X_train_normalized, y_train) # No se xq daba mejor sin normalizar

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)


Best Model: KNeighborsClassifier(algorithm='ball_tree', leaf_size=32, metric='euclidean',
                     n_neighbors=29, weights='distance')
Best AUCROC Score: 0.7648983187355279


In [161]:
from sklearn.svm import SVC

param_dist = {
    'C': scipy.stats.uniform(0, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': scipy.stats.randint(1, 6),
    'gamma': ['scale', 'auto' ], #scipy.stats.uniform(0, 1),
    'coef0': scipy.stats.uniform(0, 1),
    'shrinking': [True, False],
    'probability': [True, False],
    'tol': scipy.stats.uniform(0, 0.1),
    'cache_size': scipy.stats.randint(1, 1000),
    #'class_weight': [{0: x, 1: 1 - x} for x in np.linspace(0, 1, 10)],
}

svm = SVC()
n_iter_search = 30
random_search = RandomizedSearchCV(estimator=svm, param_distributions=param_dist, scoring='roc_auc', cv=5, n_iter=n_iter_search)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: SVC(C=5.361187448621152, cache_size=431, coef0=0.35193063363603405, degree=1,
    probability=True, tol=0.041054304536744385)
Best AUCROC Score: 0.8531367663344407


In [162]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Aca casi todas me fallan, hay que mejorarlo

param_dist = {
    #'solver': ['svd','lsqr', 'eigen'],
    #'shrinkage': ['auto',None], #scipy.stats.uniform(0, 1),
    #'priors': [],
    #'n_components': scipy.stats.randint(1, min(len(X_train[0,:]), len(X_train[0]-1))),
    'tol': scipy.stats.uniform(0, 0.1)
    #'covariance_estimator':
}


lda = LinearDiscriminantAnalysis()
n_iter_search = 30
random_search = RandomizedSearchCV(estimator=lda, param_distributions=param_dist, scoring='roc_auc', cv=5, n_iter=n_iter_search)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: LinearDiscriminantAnalysis(tol=0.0893187235489917)
Best AUCROC Score: 0.6321202053760194


In [163]:
from sklearn.naive_bayes import GaussianNB

param_dist = {
    'var_smoothing': scipy.stats.uniform(0, 1),
    'priors': [None, [0.5, 0.5]],
}

nb = GaussianNB()
n_iter_search = 30
random_search = RandomizedSearchCV(estimator=nb, param_distributions=param_dist, scoring='roc_auc', cv=5, n_iter=n_iter_search)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_score = random_search.best_score_

print("Best Model:", best_model)
print("Best AUCROC Score:", best_score)

Best Model: GaussianNB(var_smoothing=0.04293554571958946)
Best AUCROC Score: 0.8443310178193899
